In [8]:
import os
import utils_mda.seq_manipulation as seq_manipulation
from importlib import reload
reload(seq_manipulation)
import MDAnalysis
from MDAnalysis import analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pathlib import Path
import sys
from MDAnalysis.analysis import leaflet
from skspatial.objects import Line
from skspatial.objects import Points
from skspatial.plotting import plot_3d


In [51]:
import math as m

def get_vector(coord1, coord2):
    return (coord1[0]-coord2[0], coord1[0]-coord2[1], coord1[0]-coord2[2])

def vector_length(v):
    return m.sqrt(sum((c ** 2) for c in v))

def dot(v1, v2):
    return sum([c1 * c2 for c1, c2 in zip(v1, v2)])

def angle_between_vectors(v1, v2):
    return m.acos(dot(v1, v2) / (vector_length(v1) * vector_length(v2))) * (180 / m.pi)


def helix_tilt(u, peptide_name, num_peptides, membrane_type, results_folder):
    peptides = [peptide_name.split('_')[0], peptide_name.split('_')[1]] if len(peptide_name.split('_'))>2 else [peptide_name.split('_')[0]]
    pep_num_dict, _ = seq_manipulation.get_aa_sequence(u, peptides)

    box = u.dimensions
    x_memb, y_memb = box[0], box[1]
    angle_list = []
    for ts in tqdm(u.trajectory[::100]):
        L = leaflet.LeafletFinder(u, 'name P')

        z_memb = np.mean(L.groups(0).positions[:,[2]])
        memb_vector = (x_memb, y_memb, z_memb)
        

        for pep, residues in pep_num_dict.items():
            timecount1 = u.trajectory.time

            c1 = u.select_atoms(f"name CA and resid {residues[0]+1}").positions
            c2 = u.select_atoms(f"name CA and resid {residues[-1]+1}").positions
           
            pep_vector = get_vector([c1[:,[i]] for i in range (3)], [c2[:,[i]] for i in range (3)])


            tilt_angle = angle_between_vectors(pep_vector, memb_vector)
            angle_list.append((timecount1, pep , abs(90-tilt_angle)))
    df = pd.DataFrame(angle_list, columns=["Time (ns)", "Peptide_num", "Angle"])
    df['Angle']= df['Angle'].astype(float)
    df['Time (ns)'] = df['Time (ns)'].astype(float)/1000
    df['Time (ns)'] = df['Time (ns)'].astype(int)

    df.to_csv(f"{results_folder}/tilt_angle_{peptide_name}_{membrane_type}.csv")
    return df


def relative_tilt(u, peptide_name, num_peptides, membrane_type, results_folder):
    peptides = [peptide_name.split('_')[0], peptide_name.split('_')[1]] if len(peptide_name.split('_'))>2 else [peptide_name.split('_')[0]]
    pep_num_dict, seq_dict = seq_manipulation.get_aa_sequence(u, peptides)

    box = u.dimensions
    x_memb, y_memb = box[0], box[1]
    angle_list = []
    for ts in tqdm(u.trajectory[::500]):
        L = leaflet.LeafletFinder(u, 'name P')

        z_memb = np.mean(L.groups(0).positions[:,[2]])
        memb_vector = (0, 0, 1)
        
        pep_num = ''
        pep_coords = []
        for pep, residues in pep_num_dict.items():
            timecount1 = u.trajectory.time
            for res_id in residues:
                output1 = u.select_atoms(f"name CA and resid {res_id+1}").positions
                pep_coords.append(output1[0])



            points = Points(
            pep_coords,
            )

            line_fit = Line.best_fit(points)
            print(line_fit)
            pep_vector = line_fit.vector
            pep_vector = (pep_vector[0], pep_vector[1], pep_vector[2])
            tilt_angle = angle_between_vectors(pep_vector, memb_vector)
            angle_list.append((timecount1, pep , 90-tilt_angle))
    df = pd.DataFrame(angle_list, columns=["Time (ns)", "Peptide_num", "Angle"])
    df['Angle']= df['Angle'].astype(float)
    df['Time (ns)'] = df['Time (ns)'].astype(float)/1000
    df['Time (ns)'] = df['Time (ns)'].astype(int)

    df.to_csv(f"{results_folder}/tilt_angle_{peptide_name}_{membrane_type}.csv")
    return df


In [52]:

def calc_and_write_to_file(path, membrane_type, sims_type, single=None):
    results_directory = f"tilt/"
    Path(results_directory).mkdir(parents=True, exist_ok=True)
    #create selection
    all_tilt={}
    if single == "yes":
        if os.path.isdir(path):
            peptide_name = os.path.basename(path)
            print(f"Starting calculations for single peptide -- {peptide_name}")
            peptide_path = path
            u = seq_manipulation.get_universe(peptide_path)
            pep_tilt = relative_tilt(u, peptide_name, 4, membrane_type, results_directory)
            print(f"{peptide_name} --- DONE")
            all_tilt["peptide_name"] = pep_tilt
    else:
        print(f"Starting calculations for multiple peptideß")
        for directory in tqdm(os.listdir(path)):
            folder = os.path.join(path,directory)
            if os.path.isdir(folder) and f"_{membrane_type}" in os.path.basename(folder):
                peptide_path = folder
                peptide_name = os.path.basename(peptide_path)
                my_file = Path(f"{results_directory}/zpos_{peptide_name}_{membrane_type}.csv")
                if my_file.is_file():
                    print(f"Results for {peptide_name} already exists! Skipping peptide.")
                    continue
                print(f"Starting calculations for peptide -- {peptide_name}")
                u = seq_manipulation.get_universe(peptide_path)
                pep_tilt= relative_tilt(u, peptide_name, 4, membrane_type, results_directory)
                print(f"{peptide_name} --- DONE")
                all_tilt["peptide_name"] = pep_tilt
        if all_tilt:
            df_pdf = pd.concat(all_tilt, axis=1).sum(axis=1, level=0)
            df_pdf.to_csv(f"{results_directory}/tilt_ALL_{sims_type}_{membrane_type}.csv")
        else:
            sys.exit("No simulation folders found in the path provided!")
             

In [53]:

for pep in [ "WF1a_WF2"]:
    for memb in [ "pg"]:
        folder_name  = "popg" if memb == "pg" else "pepg"
        p=f"/Volumes/miru_backup/jade_2synergy/{folder_name}/{pep}_{memb}"
        calc_and_write_to_file(p, memb, "synergy", single="yes")

Starting calculations for single peptide -- WF1a_WF2_pg
ENTERED FUNCTION
FOUND TPRrS


  0%|          | 0/101 [00:00<?, ?it/s]

found U
['WF1a', 'WF2']
peptide_list2 ['WF1a', 'WF1a', 'WF1a', 'WF1a', 'WF2', 'WF2', 'WF2', 'WF2']
Line(point=Point([17.458   , 55.191505, 99.630005], dtype=float32), direction=Vector([-0.9922298 ,  0.04776229,  0.1148859 ], dtype=float32))
Line(point=Point([ 20.88625 ,  63.437004, 100.74151 ], dtype=float32), direction=Vector([-0.99585223, -0.08767722,  0.02431105], dtype=float32))
Line(point=Point([ 38.754833,  49.497173, 100.398026], dtype=float32), direction=Vector([-0.84526896,  0.53319967,  0.03490753], dtype=float32))
Line(point=Point([35.447502, 42.738884, 99.851395], dtype=float32), direction=Vector([-0.82184327,  0.56827503,  0.04046062], dtype=float32))
Line(point=Point([47.82676 , 46.740677, 98.80173 ], dtype=float32), direction=Vector([-0.9934583 ,  0.09827595,  0.05815977], dtype=float32))


  1%|          | 1/101 [00:00<00:31,  3.16it/s]

Line(point=Point([48.231304, 48.504467, 99.04302 ], dtype=float32), direction=Vector([-0.9938282 ,  0.09521034,  0.05692583], dtype=float32))
Line(point=Point([45.401863, 54.349503, 99.85493 ], dtype=float32), direction=Vector([-0.92936873,  0.36147964,  0.07487503], dtype=float32))
Line(point=Point([47.22005, 56.81607, 99.77281], dtype=float32), direction=Vector([-0.8717317 ,  0.48462927,  0.07223779], dtype=float32))
Line(point=Point([ 40.489998,  58.109997, 100.102   ], dtype=float32), direction=Vector([-0.9878439 ,  0.11178992,  0.10801613], dtype=float32))
Line(point=Point([ 31.231247,  65.68025 , 101.27074 ], dtype=float32), direction=Vector([-0.98047996,  0.16940156,  0.09981046], dtype=float32))


  2%|▏         | 2/101 [00:00<00:32,  3.09it/s]

Line(point=Point([ 46.946667,  50.951504, 100.393   ], dtype=float32), direction=Vector([-0.82716084,  0.55759865,  0.06991894], dtype=float32))
Line(point=Point([ 40.8115  ,  43.155876, 100.786125], dtype=float32), direction=Vector([-0.8430272 ,  0.5318884 ,  0.07999926], dtype=float32))
Line(point=Point([ 50.18924 ,  46.423424, 100.282776], dtype=float32), direction=Vector([-0.9678481 ,  0.23719239,  0.08372433], dtype=float32))
Line(point=Point([ 50.94854 ,  47.965454, 100.02845 ], dtype=float32), direction=Vector([-0.9700441 ,  0.22819354,  0.08332007], dtype=float32))
Line(point=Point([46.742455, 50.84225 , 99.78211 ], dtype=float32), direction=Vector([-0.8874895 ,  0.45697683,  0.0594521 ], dtype=float32))
Line(point=Point([48.591732, 51.355446, 99.428665], dtype=float32), direction=Vector([-0.5977123 ,  0.8010679 ,  0.03209737], dtype=float32))
Line(point=Point([ 18.005001,  57.336006, 100.470505], dtype=float32), direction=Vector([-0.99152136,  0.08689226,  0.09661849], dtype=f

  3%|▎         | 3/101 [00:01<00:34,  2.86it/s]

Line(point=Point([ 47.656143,  48.552692, 102.21708 ], dtype=float32), direction=Vector([-0.9909415 ,  0.12921853,  0.03657193], dtype=float32))
Line(point=Point([ 43.668255,  52.32381 , 102.15572 ], dtype=float32), direction=Vector([-0.94248533,  0.33329883,  0.02516611], dtype=float32))
Line(point=Point([ 45.681717,  52.702274, 101.9781  ], dtype=float32), direction=Vector([-0.8576224 ,  0.5136816 ,  0.02480049], dtype=float32))


KeyboardInterrupt: 